In [2]:
!pip3 install fancyimpute

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 7.7 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29879 sha256=974de53b398c9bbf4227a2b3818e062d468ececbb77a7726cb809a89af75c76f
  Stored in directory: /root/.cache/pip/wheels/1a/f3/a1/f7f10b5ae2c2459398762a3fcf4ac18c325311c7e3163d5a15
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11331 sha256=e406e2c5c3db267d09899f2dfcd9aa8fc22a5371a174def3f002d3bb98329a95
  Stored in directory: /root/.cache/pip/wheels/ea/e8/e0/79872972161e54486517ae507f94b2c7cea27fb7ef793bd415
Successfully built fancyimpute knnimpute


In [1]:
from google.colab import auth
from google.colab import drive
import pandas as pd
from pathlib import Path
import numpy as np
from scipy.io import loadmat
from scipy.spatial.distance import cdist
from scipy.interpolate import interp1d
from scipy import stats

import argparse

from fancyimpute import KNN

ModuleNotFoundError: No module named 'fancyimpute'

In [4]:
# Authenticate account to use credentials
auth.authenticate_user()

In [5]:
# Connecting root Google Drive to CoLab and mounting the drive
if 'google.colab' in str(get_ipython()):
    root = '/content/drive'
    # print(root)
    drive.mount(root)

# Exits operation if not running on Colab
else:
    print('Not running on CoLab')

Mounted at /content/drive


In [6]:
# Connecting my own directory to where the file is located
if 'google.colab' in str(get_ipython()):
    project_path = Path(root) / 'MyDrive' / 'MIMIC_Tables'
else:
    project_path = Path()



In [7]:
parser = argparse.ArgumentParser(...)
parser.add_argument("--process_raw", action="store_true", help="If specified, additionally save trajectories without normalized features")
parser.add_argument("--save_intermediate", action="store_true",  help="If specified, save off intermediate tables used to construct final patient table")

pargs, _unknown = parser.parse_known_args()

In [8]:
# Importing all data tables

antibiotics_df = pd.read_csv(project_path / 'antibiotics.csv')

culture_df = pd.read_csv(project_path / 'chartevents.csv')

microbio_df = pd.read_csv(project_path / 'microbioevents.csv')
demographics_df = pd.read_csv(project_path / 'demographics.csv')
ce010 = pd.read_csv(project_path / 'vitals010000.csv', sep = '|')
ce1020 = pd.read_csv(project_path / 'vitals1000020000.csv', sep = '|')
ce2030 = pd.read_csv(project_path / 'vitals2000030000.csv', sep = '|')
ce3040 = pd.read_csv(project_path / 'vitals3000040000.csv', sep = '|')
ce4050 = pd.read_csv(project_path / 'vitals4000050000.csv', sep = '|')
ce5060 = pd.read_csv(project_path / 'vitals5000060000.csv', sep = '|')
ce6070 = pd.read_csv(project_path / 'vitals6000070000.csv', sep = '|')
ce7080 = pd.read_csv(project_path / 'vitals7000080000.csv', sep = '|')
ce8090 = pd.read_csv(project_path / 'vitals8000090000.csv', sep = '|')
ce90100 = pd.read_csv(project_path / 'vitals90000100000.csv', sep = '|')
labU = [pd.read_csv(project_path / 'ce_labs.csv', sep = '|'), pd.read_csv(project_path / 'le_labs.csv', sep = '|')]
mechvent_df = pd.read_csv(project_path / 'mechvent.csv', sep = '|')
preadm_fluids_df = pd.read_csv(project_path / 'preadm_fluids.csv', sep = '|')
mv_fluids_df = pd.read_csv(project_path / 'metavision_fluids.csv')
cv_fluids_df = pd.read_csv(project_path / 'carevue_fluids.csv', sep = '|')
mv_vaso_df = pd.read_csv(project_path / 'metavision_vasopressors.csv', sep ='|')
cv_vasp_df = pd.read_csv(project_path / 'carevue_vasopressors.csv', sep ='|')
preadm_output_df = pd.read_csv(project_path / 'preadm_output.csv', sep ='|')
output_df = pd.read_csv(project_path / 'output.csv', sep = '|')


In [9]:
labU[1].rename(columns = {'timestp':'charttime'}, inplace = True)
labU = pd.concat(labU, sort=False, ignore_index=True)

In [10]:
microbio_df['charttime'] = microbio_df['charttime'].fillna(microbio_df['chartdate'])
del microbio_df['chartdate']
bacterio = pd.concat([microbio_df, culture_df], sort = False, ignore_index = True)

demographics_df['morta_90'].fillna(0, inplace = True)
demographics_df['morta_hosp'].fillna(0, inplace = True)
demographics_df['elixhauser'].fillna(0, inplace = True)

<ipython-input-10-b0b1252ff580>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  demographics_df['morta_90'].fillna(0, inplace = True)
<ipython-input-10-b0b1252ff580>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [11]:
# Keep only the first icustay of an admission
demographics_df = demographics_df.drop_duplicates(subset = ['admittime', 'dischtime'], keep='first')

In [12]:
# Get list of all icustayids to iterate over for the rest of the script
icustayidlist = list(demographics_df.icustay_id.values)

In [13]:
# Grouping all discharge times for each patient and using them in a comparison
subj_dischtime_list = demographics_df.sort_values(by='admittime').groupby('subject_id').apply(lambda df: np.unique(df.dischtime.values)) # List of discharge times for each patient (dict, key = subject_id)

<ipython-input-13-3de129ea0cda>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  subj_dischtime_list = demographics_df.sort_values(by='admittime').groupby('subject_id').apply(lambda df: np.unique(df.dischtime.values)) # List of discharge times for each patient (dict, key = subject_id)


In [14]:
def determine_readmission(s, dischtimes=subj_dischtime_list, cutoff=3600*24*30):
    '''
    Evaluates each row of the dataframe and chooses whether the current admission occurs within cutoff of previous discharge
    (here, cutoff=30 days is the default)
    '''
    subject, admission, discharge = s[['subject_id','admittime','dischtime']]

    # Check for readmission
    subj_stay_idx = np.where(dischtimes[subject] == discharge)[0][0]
    s['re_admission'] = 0
    if subj_stay_idx > 0:
      if(admission - dischtimes[subject][subj_stay_idx-1]) <= cutoff:
        s['re_admission'] = 1

    return s

# Apply above function to determine readmissions
demographics_df = demographics_df.apply(determine_readmission, axis=1)

In [15]:
# Other helper functions

def SAH (input, vitalslab_hold, adjust=0):
  '''
  Komorowski(2017) copied value from previous cell if missing value are within the hold period for this variable
  (e.g. 48h for weight, 2h for HR)
  vitalslab_hold = 2x55 cell (with row1 = strings of names; row 2 hold time)

  '''
  temp = np.copy(input)
  hold = vitalslab_hold.values[0, :]
  nrow, ncol = temp.shape

  lastcharttime = np.zeros(ncol)
  lastvalue = np.zeros(ncol)
  oldstayid = temp[0,1]

  for i in range(3+adjust,ncol):
    for j in range(nrow):
      if oldstayid != temp[j,1]:
        lastcharttime = np.zeros(ncol)
        lastvalue = np.zeros(ncol)
        oldstayid = temp[j, 1]
      if not np.isnan(temp[j, i]):
        lastcharttime[i] = temp[j, 2]
        lastvalue[i] = temp[j, i]
      if j > 0:
        if (np.isnan(temp[j, i])) and (temp[j, 1] == oldstayid) and ((temp[j, 2] - lastcharttime[i]) <= hold[i-(3+adjust)]*3600):
          temp[j,i] = lastvalue[i]
  return temp

In [16]:
def fixgaps(x):
  '''
  Linear interpolation of gaps in timeseries
  YOUT=FIXGAPS(YIN) linearly interpolates over NaN in the timeseries input,
  ignores trailing and leading NaNs
  '''
  y = np.copy(x)
  bd = np.isnan(x)
  gd = np.arange(len(x))[~bd]
  bd[:min(gd)] = False
  bd[max(gd)+1:] = False
  y[bd] = interp1d(gd,x[gd])(np.arange(len(x))[bd])
  return y

def deloutabove(a, col_no, a_max):
  a[a[:,col_no] > a_max, col_no] = np.nan
  return a

def deloutbelow(a, col_no, a_min):
  a[a[:,col_no] < a_min, col_no] = np.nan
  return a

In [17]:
# Normalized infusion rates
# If 100 mL of hypertonic fluid (600 mosm/l) is given at 100ml/h (given in 1h) it is 200mL of NS equivalent

mv_fluids_df['norm_rate_of_infusion'] = mv_fluids_df['tev']*mv_fluids_df['rate']/mv_fluids_df['amount']

print('Filling-in missing ICUSTAY IDs in bacterio')

for i in bacterio.index.tolist():
   if np.isnan(bacterio.loc[i, 'icustay_id']):
        o = bacterio.loc[i, 'charttime']
        subjectid = bacterio.loc[i, 'subject_id']
        hadmid = bacterio.loc[i, 'hadm_id']
        ii = demographics_df.index[demographics_df['subject_id'] == subjectid].tolist()
        jj = demographics_df.index[(demographics_df['subject_id'] == subjectid) & (demographics_df['hadm_id'] == hadmid)].tolist()
        for j in range(len(ii)):
            if (o >= demographics_df.loc[ii[j], 'intime'] - 48*3600) and (o <= demographics_df.loc[ii[j], 'outtime'] + 48*3600):
                bacterio.loc[i,'icustay_id'] = demographics_df.loc[ii[j], 'icustay_id']
            elif len(ii)==1:   # If we cant confirm from admission and discharge time but there is only 1 admission: it's the one!!
                bacterio.loc[i,'icustay_id'] = demographics_df.loc[ii[j], 'icustay_id']

print('Filling-in missing ICUSTAY IDs in bacterio - 2')

for i in bacterio.index.tolist():
    if np.isnan(bacterio.loc[i, 'icustay_id']):
        subjectid = bacterio.loc[i, 'subject_id']
        hadmid = bacterio.loc[i, 'hadm_id']
        jj = demographics_df.index[(demographics_df['subject_id'] == subjectid) & (demographics_df['hadm_id'] == hadmid)].tolist()
        if len(jj) == 1:
            bacterio.loc[i,'icustay_id'] = demographics_df.loc[jj[0], 'icustay_id']


print('Filling-in missing ICUSTAY IDs in antibiotics_df')

for i in antibiotics_df.index.tolist():
    if np.isnan(antibiotics_df.loc[i,'icustay_id']):
        o = antibiotics_df.loc[i,'startdate']  #time of event
        hadmid = antibiotics_df.loc[i,'hadm_id']
        ii     = demographics_df.index[demographics_df['hadm_id'] == hadmid].tolist()
        for j in range(len(ii)):
            if o >= demographics_df.loc[ii[j],'intime'] - 48*3600 and o <= demographics_df.loc[ii[j], 'outtime'] + 48*3600:
                antibiotics_df.loc[i, 'icustay_id'] = demographics_df.loc[ii[j], 'icustay_id']
            elif len(ii) == 1:   #if we cant confirm from admission and discharge time but there is only 1 admission: it's the one!!
                antibiotics_df.loc[i, 'icustay_id'] = demographics_df.loc[ii[j], 'icustay_id']

Filling-in missing ICUSTAY IDs in bacterio
Filling-in missing ICUSTAY IDs in bacterio - 2
Filling-in missing ICUSTAY IDs in antibiotics_df


In [18]:
# Finding presumed onset of infection

onset = dict()
num_onset = 0

for icustayid in icustayidlist:
    onset[icustayid] = np.zeros(3)
    ab = antibiotics_df.loc[antibiotics_df['icustay_id'] == icustayid, 'startdate']   # Start time of abx for this icustayid
    bact = bacterio.loc[bacterio['icustay_id'] == icustayid, 'charttime']   # Time of sample
    subj_bact = bacterio.loc[bacterio['icustay_id'] == icustayid,'subject_id']

    if len(ab) > 0 and len(bact) > 0:   # If we have data for both: proceed
        # Pairwise distances between antibiotic adminstration and requested cultures, in hours
        D = cdist(ab.values.reshape(ab.values.shape[0],1),bact.values.reshape(bact.values.shape[0],1))/3600
        for i in range(D.shape[0]):  #looping through all rows of adminsitered antibiotics, from early to late
            M, I = np.min(D[i,:]), np.argmin(D[i,:])        # minimum distance in this row
            ab1 = ab.iloc[i]       # timestamp of this value in list of antibiotics
            bact1 = bact.iloc[I]   # timestamp in list of cultures
            if M <= 24 and ab1 <= bact1:      # if ab was first and delay < 24h
                onset[icustayid][0] = subj_bact.iloc[0]
                onset[icustayid][1] = icustayid
                onset[icustayid][2] = ab1     # Onset of infection = abx time
                num_onset += 1
                break
            elif M <= 72 and ab1 >= bact1:    # elseif sample was first and delay < 72h
                onset[icustayid][0] = subj_bact.iloc[0]
                onset[icustayid][1] = icustayid
                onset[icustayid][2] = bact1       # Onset of infection = sample time
                num_onset += 1
                break

In [19]:
print('Full ICU -- Number of preliminary, presumed septic trajectories: ', num_onset)

Full ICU -- Number of preliminary, presumed septic trajectories:  25367


In [20]:
# Replace itemid in labs with column number
# This will accelerate process later

Reflabs = pd.read_csv(project_path /"Reflabs.tsv", sep = '\t', header=None)
Reflabs_values = np.unique(Reflabs.fillna(-10000))[1:]
Reflabs_id_dict = {}
for r in Reflabs_values:
    try:
        Reflabs_id_dict[r] = np.max(np.where(Reflabs.values == r)[0]) + 1 # for row: +1 due to Index correction python
    except:
        print(r)
        break
itemid_col = labU.columns.tolist().index('itemid')
labU_temp = labU.values
for c in range(labU_temp.shape[0]):
    labU_temp[c,itemid_col] = Reflabs_id_dict[labU_temp[c,itemid_col]]
for i, c in enumerate(labU.columns.tolist()):
    labU.loc[:,c] = labU_temp[:,i]

In [21]:
# Replace itemid in vitals with column number

Refvitals = pd.read_csv(project_path /"Refvitals.tsv", sep = '\t', header=None)
Refvitals_values = np.unique(Refvitals.fillna(-10000))[1:]
Refvitals_id_dict = {}
for r in Refvitals_values:
    Refvitals_id_dict[r] = np.max(np.where(Refvitals.values == r)[0]) + 1 # +1 due to index correction for Python from MATLAB
ce_dfs = [ce010, ce1020, ce2030, ce3040, ce4050, ce5060, ce6070, ce7080, ce8090, ce90100]
for ce_df in ce_dfs:
    itemid_col = ce_df.columns.tolist().index('itemid')
    ce_df_temp = ce_df.values
    for c in range(ce_df_temp.shape[0]):
        ce_df_temp[c,itemid_col] = Refvitals_id_dict[ce_df_temp[c,itemid_col]]
    for i, c in enumerate(ce_df.columns.tolist()):
        ce_df.loc[:,c] = ce_df_temp[:,i]

In [22]:
print(mechvent_df.columns)

Index(['icustay_id', 'charttime', 'MechVent', 'Extubated', 'SelfExtubated'], dtype='object')


In [23]:
reformat = np.nan*np.ones((2000000,68))  # Final table
qstime = dict()
winb4 = 25   # Lower limit for inclusion of data (24h before time flag)
winaft = 49  # Upper limit (48h after)
irow = 0  # Recording row for summary table

for icustayid in icustayidlist:
    qstime[icustayid] = np.zeros(4)
    qst = onset[icustayid][2] #flag for presumed infection
    if qst > 0:  # if we have a flag
        d1 = demographics_df.loc[demographics_df['icustay_id'] == icustayid, ['age', 'dischtime']].values[0] # Age of patient + discharge time
        if d1[0] > 6574:  # If older than 18 years old
            # CHARTEVENTS
            if (icustayid-200000) < 10000:
                temp=ce010
            elif (icustayid-200000) < 20000:
                temp=ce1020
            elif (icustayid-200000) < 30000:
                temp=ce2030
            elif (icustayid-200000) < 40000:
                temp=ce3040
            elif (icustayid-200000) < 50000:
                temp=ce4050
            elif (icustayid-200000) < 60000:
                temp=ce5060
            elif (icustayid-200000) < 70000:
                temp=ce6070
            elif (icustayid-200000) < 80000:
                temp=ce7080
            elif (icustayid-200000) < 90000:
                temp=ce8090
            else:
                temp=ce90100
            temp = temp[temp['icustay_id'] == icustayid]

            ii = (temp['charttime'] >= qst - (winb4+4)*3600) & (temp['charttime'] <= qst + (winaft+4)*3600) # Time period of interest -4h and +4h
            temp = temp.loc[ii]   # Only time period of interest

            # LABEVENTS
            ii = labU['icustay_id'] == icustayid
            temp2 = labU.loc[ii]
            ii = (temp2['charttime'] >= qst - (winb4+4)*3600) & (temp2['charttime'] <= qst + (winaft+4)*3600) # Time period of interest -4h and +4h
            temp2 = temp2.loc[ii]   # Only time period of interest

            # Mech Vent + ?extubated
            ii = mechvent_df['icustay_id'] == icustayid
            temp3 = mechvent_df.loc[ii]
            ii = (temp3['charttime'] >= qst - (winb4+4)*3600) & (temp3['charttime'] <= qst + (winaft+4)*3600) # Time period of interest -4h and +4h
            temp3 = temp3.loc[ii]   #only time period of interest
            t = np.unique(pd.concat([temp['charttime'], temp2['charttime'], temp3['charttime']], ignore_index=True).values) # List of unique timestamps from all 3 sources / sorted in ascending order

            if len(t) > 0:
                for i in range(len(t)):
                    #CHARTEVENTS
                    ii = temp['charttime'] == t[i]
                    col = temp.loc[ii,'itemid']
                    value = temp.loc[ii,'valuenum']
                    reformat[irow, 0] = i+1 #timestep
                    reformat[irow, 1] = icustayid
                    reformat[irow, 2] = t[i] #charttime
                    reformat[irow, 2+col.astype(int).values] = value.values # Store available values

                    # LAB VALUES
                    ii = temp2['charttime'] == t[i]
                    col = temp2.loc[ii, 'itemid']
                    value = temp2.loc[ii, 'valuenum']
                    reformat[irow, 30+col.astype(int).values] = value.values  # Store available values

                    # Mechanical Ventilation
                    ii = temp3['charttime'] == t[i]
                    if np.nansum(ii) > 0:
                        col = temp3.loc[ii, 'MechVent']
                        value = temp3.loc[ii, 'Extubated']
                        reformat[irow, 66] = col.values[0] # Store available values
                        reformat[irow, 67] = value.values[0] # Store available values
                    else:
                        reformat[irow, 66]= np.nan
                        reformat[irow, 67]= np.nan
                    irow += 1

                qstime[icustayid][0] = qst # Flag for presumed infection / this is time of sepsis if SOFA >=2 for this patient
                # SAVE FIRST and LAST TIMESTAMPS, in QSTIME, for each ICUSTAYID
                qstime[icustayid][1] = t[0]   # First timestamp
                qstime[icustayid][2] = t[-1]  # Last timestamp
                qstime[icustayid][3] = d1[1]  # Discharge time

reformat = np.delete(reformat, range(irow, len(reformat)) ,axis=0)  # Delete unused rows

In [24]:
# Outliers
reformat = deloutabove(reformat, 4, 300)

# Heart Rate
reformat = deloutabove(reformat, 7, 250)

# Blood Pressure
reformat = deloutabove(reformat, 8, 300)
reformat = deloutbelow(reformat, 9, 0)
reformat = deloutabove(reformat, 9, 200)
reformat = deloutbelow(reformat, 10, 0)
reformat = deloutabove(reformat, 10, 200)

# Respiratory Rate
reformat = deloutabove(reformat, 11, 80)

# SpO2
reformat = deloutabove(reformat, 12, 150)
reformat[reformat[:, 12]>100, 12] = 100

# Temperature
reformat[(reformat[:, 13] > 90) & (np.isnan(reformat[:, 14])), 14] = reformat[(reformat[:, 13] > 90) & (np.isnan(reformat[:, 14])), 13]
reformat = deloutabove(reformat, 13, 90)

# Interface / is in col 22
# FiO2
reformat = deloutabove(reformat, 22, 100)
reformat[reformat[:, 22] < 1 , 22] = reformat[reformat[:,22] < 1 , 22]*100

reformat = deloutbelow(reformat, 22, 20)
reformat = deloutabove(reformat, 23, 1.5)

# O2 FLOW
reformat = deloutabove(reformat, 24, 70)

#PEEP
reformat=deloutbelow(reformat, 25, 0)
reformat=deloutabove(reformat, 25, 40)

# Total Volume
reformat=deloutabove(reformat, 26, 1800)

# Mean Volume
reformat=deloutabove(reformat, 27, 50)

# Potassium
reformat=deloutbelow(reformat, 31, 1)
reformat=deloutabove(reformat, 31, 15)

# Sodium
reformat=deloutbelow(reformat, 32, 95)
reformat=deloutabove(reformat, 32, 178)

# Chloride
reformat=deloutbelow(reformat, 33, 70)
reformat=deloutabove(reformat, 33, 150)

# Glucose
reformat=deloutbelow(reformat, 34, 1)
reformat=deloutabove(reformat, 34, 1000)

# Creatinine
reformat=deloutabove(reformat, 36, 150)

# Magnesium
reformat=deloutabove(reformat, 37, 10)

# Calcium
reformat=deloutabove(reformat, 38, 20)

# Ionized Calcium
reformat=deloutabove(reformat, 39, 5)

# CO2
reformat=deloutabove(reformat, 40, 120)

# SGPT/SGOT
reformat=deloutabove(reformat, 41, 10000)
reformat=deloutabove(reformat, 42, 10000)

# Hb/Ht
reformat=deloutabove(reformat, 49, 20)
reformat=deloutabove(reformat, 50, 65)

# White Blood Cells
reformat=deloutabove(reformat, 52, 500)

# Platelets
reformat=deloutabove(reformat, 53, 2000)

# INR
reformat=deloutabove(reformat, 57, 20)

# pH
reformat=deloutbelow(reformat, 58, 6.7)
reformat=deloutabove(reformat, 58, 8)

# pO2
reformat=deloutabove(reformat, 59, 700)

# pCO2
reformat=deloutabove(reformat, 60, 200)

# Base Excess
reformat=deloutbelow(reformat, 61, -50)

# Lactate
reformat=deloutabove(reformat, 62, 30)



In [25]:
# Estimate GCS from RASS - data from Wesley JAMA 2003
reformat[(np.isnan(reformat[:, 5])) & (reformat[:, 6] >= 0), 5] = 15
reformat[(np.isnan(reformat[:, 5])) & (reformat[:, 6] == -1), 5] = 14
reformat[(np.isnan(reformat[:, 5])) & (reformat[:, 6] == -2), 5] = 12
reformat[(np.isnan(reformat[:, 5])) & (reformat[:, 6] == -3), 5] = 11
reformat[(np.isnan(reformat[:, 5])) & (reformat[:, 6] == -4), 5] = 6
reformat[(np.isnan(reformat[:, 5])) & (reformat[:, 6] == -5), 5] = 3

# FiO2
reformat[(~np.isnan(reformat[:, 22])) & (np.isnan(reformat[:, 23])), 23] = reformat[(~np.isnan(reformat[:, 22])) & (np.isnan(reformat[:, 23])), 22] / 100
reformat[(~np.isnan(reformat[:, 23])) & (np.isnan(reformat[:, 22])), 22] = reformat[(~np.isnan(reformat[:, 23])) & (np.isnan(reformat[:, 22])), 23] * 100


sample_and_hold = pd.read_csv(project_path/'sample_and_hold.csv', index_col = None)

reformatsah = SAH(reformat,sample_and_hold)  # Do SAH first to handle this task

# NO FiO2, YES O2 flow, no interface OR cannula
ii = np.where((np.isnan(reformatsah[:, 22])) & (~np.isnan(reformatsah[:, 24])) & ((reformatsah[:, 21] == 0) | (reformatsah[:, 21] == 2)))[0] #As np.where returns a tuple
reformat[ii[reformatsah[ii, 24] <= 15], 22] = 70
reformat[ii[reformatsah[ii, 24] <= 12], 22] = 62
reformat[ii[reformatsah[ii, 24] <= 10], 22] = 55
reformat[ii[reformatsah[ii, 24] <= 8], 22]  = 50
reformat[ii[reformatsah[ii, 24] <= 6], 22]  = 44
reformat[ii[reformatsah[ii, 24] <= 5], 22]  = 40
reformat[ii[reformatsah[ii, 24] <= 4], 22]  = 36
reformat[ii[reformatsah[ii, 24] <= 3], 22]  = 32
reformat[ii[reformatsah[ii, 24] <= 2], 22]  = 28
reformat[ii[reformatsah[ii, 24] <= 1], 22]  = 24

# NO FiO2, NO O2 flow, no interface OR cannula
ii = np.where((np.isnan(reformatsah[:, 22])) & np.isnan(reformatsah[:, 24]) & ((reformatsah[:, 21] == 0) | (reformatsah[:, 21] == 2)))[0]  #no fio2 given and o2flow given, no interface OR cannula
reformat[ii, 22] = 21

# NO FiO2, YES O2 flow, face mask OR.... OR ventilator (assume it's face mask)
ii = np.where((np.isnan(reformatsah[:, 22])) & (~np.isnan(reformatsah[:, 24])) &
((reformatsah[:, 21] == 1) | (reformatsah[:, 21]==3) | (reformatsah[:, 21] == 4) | (reformatsah[:, 21] == 5) | (reformatsah[:, 21]==6) | (reformatsah[:, 21]==9) | (reformatsah[:, 21]==10)))[0]
reformat[ii[reformatsah[ii, 24]<=15], 22] = 75
reformat[ii[reformatsah[ii, 24]<=12], 22] = 69
reformat[ii[reformatsah[ii, 24]<=10], 22] = 66
reformat[ii[reformatsah[ii, 24]<=8], 22]  = 58
reformat[ii[reformatsah[ii, 24]<=6], 22]  = 40
reformat[ii[reformatsah[ii, 24]<=4], 22]  = 36

# NO FiO2, NO O2 flow, face mask OR ....OR ventilator
ii = np.where(np.isnan(reformatsah[:, 22]) & np.isnan(reformatsah[:, 24]) & ((reformatsah[:, 21] == 1) | (reformatsah[:, 21] == 3) |
(reformatsah[:, 21] == 4) | (reformatsah[:, 21] == 5) | (reformatsah[:, 21] == 6) | (reformatsah[:, 21] == 9) | (reformatsah[:, 21] == 10)))[0]  #no fio2 given and o2flow given, no interface OR cannula
reformat[ii, 22] = np.nan

# NO FiO2, YES O2 flow, Non rebreather mask
ii = np.where(np.isnan(reformatsah[:, 22]) & (~np.isnan(reformatsah[:, 24])) & (reformatsah[:, 21] == 7))[0]
reformat[ii[reformatsah[ii, 24] >= 10], 22] = 90
reformat[ii[reformatsah[ii, 24] >= 15], 22] = 100
reformat[ii[reformatsah[ii, 24] < 10], 22]  = 80
reformat[ii[reformatsah[ii, 24] <= 8], 22]  = 70
reformat[ii[reformatsah[ii, 24] <= 6], 22]  = 60

# NO FiO2, NO O2 flow, NRM
ii= np.where(np.isnan(reformatsah[:, 22]) & np.isnan(reformatsah[:, 24]) & (reformatsah[:, 21]==7))[0]  #no fio2 given and o2flow given, no interface OR cannula
reformat[ii, 22] = np.nan

# Update FiO2 columns again
ii = (~np.isnan(reformat[:, 22])) & (np.isnan(reformat[:,23]))
reformat[ii, 23] = reformat[ii, 22]/100
ii = (~np.isnan(reformat[:, 23])) & (np.isnan(reformat[:, 22]))
reformat[ii, 22] = reformat[ii, 23]*100

# Blood Pressure
ii = (~np.isnan(reformat[:, 8])) & (~np.isnan(reformat[:, 9])) & np.isnan(reformat[:, 10])
reformat[ii, 10] = (3*reformat[ii, 9] - reformat[ii, 8])/2
ii = (~np.isnan(reformat[:, 8])) & (~np.isnan(reformat[:, 10])) & np.isnan(reformat[:, 9])
reformat[ii, 9] = (reformat[ii, 8] + 2*reformat[ii, 10])/3
ii = (~np.isnan(reformat[:, 9])) & (~np.isnan(reformat[:, 10])) & np.isnan(reformat[:, 8])
reformat[ii, 8] = 3*reformat[ii, 9] - 2*reformat[ii, 10]

# Temperature
# Some values recorded in the wrong column
ii = (reformat[:, 14] > 25) & (reformat[:, 14] < 45) # tempF close to 37deg??!
reformat[ii, 13] = reformat[ii, 14]
reformat[ii, 14] = np.nan
ii = reformat[:, 13] >70  # tempC > 70, likely recorded in Farenheit
reformat[ii, 14] = reformat[ii, 13]
reformat[ii, 13] = np.nan
ii = (~np.isnan(reformat[:, 13])) & np.isnan(reformat[:, 14])
reformat[ii, 14] = reformat[ii, 13]*1.8+32
ii = (~np.isnan(reformat[:, 14])) & np.isnan(reformat[:, 13])
reformat[ii, 13] = (reformat[ii, 14] - 32)/1.8

# Hb/Ht
ii = (~np.isnan(reformat[:,49])) & np.isnan(reformat[:, 50])
reformat[ii, 50] = (reformat[ii, 49] * 2.862) + 1.216
ii = (~np.isnan(reformat[:, 50])) & np.isnan(reformat[:, 49])
reformat[ii, 49] = (reformat[ii, 50] - 1.216)/2.862

# Bilirubin
ii = (~np.isnan(reformat[:, 43])) & np.isnan(reformat[:, 44])
reformat[ii, 44] = (reformat[ii, 43]*0.6934)-0.1752
ii = (~np.isnan(reformat[:, 44])) & np.isnan(reformat[:, 43])
reformat[ii, 43] = (reformat[ii, 44] + 0.1752)/0.6934

In [26]:
# Sample and Hold on raw data

reformat = SAH(reformat[:,0:68],sample_and_hold)

In [27]:
print(mv_fluids_df.columns.tolist())

['icustay_id', 'starttime', 'endtime', 'itemid', 'amount', 'rate', 'tev', 'norm_rate_of_infusion']


In [28]:
timestep = 4  # Resolution of timesteps, in 4 hour blocks
irow = 0
icustayidlist = np.unique(reformat[:,1]).astype(np.int32)
reformat2 = np.nan*np.ones((reformat.shape[0], 85))  # Output array
num_patients = len(icustayidlist)  # Number of patients
# Adding 2 empty cols for future shock index=HR/SBP and P/F
reformat = np.hstack([reformat, np.nan*np.ones((reformat.shape[0], 2))])

for i in range(len(icustayidlist)):
    icustayid = icustayidlist[i]

    #CHARTEVENTS AND LAB VALUES
    temp = reformat[reformat[:,1] == icustayid,:]   # subtable of interest
    beg = temp[0,2]   # timestamp of first record

    #IV FLUID STUFF
    iv = mv_fluids_df['icustay_id'] == icustayid         # rows of interest in inputMV
    input = mv_fluids_df[iv]                             # subset of interest
    iv = cv_fluids_df['icustay_id'] == icustayid         # rows of interest in inputCV
    input2 = cv_fluids_df[iv]                            # subset of interest
    startt = input['starttime']                     # start of all infusions and boluses
    endt = input['endtime']                         # end of all infusions and boluses
    rate = input['norm_rate_of_infusion']           # rate of infusion (is NaN for boluses) || corrected for tonicity
    pread = preadm_fluids_df[preadm_fluids_df['icustay_id'] == icustayid]['inputpreadm'] # preadmission volume

    input = input.values
    input2 = input2.values

    if len(pread) >0:  # store the value, if available
        totvol = np.nansum(pread)
    else:
        totvol=0   # if not documented: it's zero

    # Compute volume of fluid given before start of record!!!
    t0 = 0
    t1 = beg
    # Input from MetaVision (4 ways to compute)
    infu = np.nansum(rate*(endt-startt)*((endt<=t1) & (startt >= t0))/3600 + rate*(endt-t0)*((startt <= t0) & (endt <= t1) & (endt >= t0))/3600 +
                            rate*(t1-startt)*((startt >= t0) & (endt >=t1) & (startt <= t1))/3600 + rate*(t1-t0)*((endt >= t1) & (startt <= t0))/3600)
    # All boluses received during this timestep, from inputMV (need to check rate is NaN) and inputCV (simpler):
    bolus = np.nansum(input[np.isnan(input[:, 5]) & (input[:, 1] >= t0) & (input[:, 1] <= t1), 6]) + np.nansum(input2[(input2[:, 1] >= t0) & (input2[:, 1] <= t1), 4])
    totvol = np.nansum([totvol,infu,bolus])


    #VASOPRESSORS
    iv = mv_vaso_df['icustay_id'] == icustayid  # rows of interest in vasoMV
    vaso1 = mv_vaso_df[iv]    # subset of interest
    iv = cv_vasp_df['icustay_id'] == icustayid   # rows of interest in vasoCV
    vaso2 = cv_vasp_df[iv]    # subset of interest
    startv = vaso1['starttime'].values  # start of VP infusion
    endv = vaso1['endtime'].values      # end of VP infusions
    ratev = vaso1['rate_std'].values  # rate of VP infusion

    # DEMOGRAPHICS / gender, age, elixhauser, re-admit, died in hosp?, died within
    # 48h of out_time (likely in ICU or soon after), died within 90d after admission?
    demogi = demographics_df['icustay_id'] == icustayid
    dem = np.array(
            list(demographics_df.gender[demogi].values) +
            list(demographics_df.age[demogi].values) +
            list(demographics_df.elixhauser[demogi].values) +
            list(demographics_df.re_admission[demogi].values) +
            list(demographics_df.morta_hosp[demogi].values) +
            list(abs(demographics_df.dod[demogi].values - demographics_df.outtime[demogi].values) < (24*3600*2)) +
            list(demographics_df.morta_90[demogi].values) +
            [(qstime[icustayid][3] - qstime[icustayid][2])/3600]
            )


    #URINE OUTPUT
    iu = output_df['icustay_id'] == icustayid   #rows of interest in inputMV
    output = output_df[iu]    #subset of interest
    pread = preadm_output_df[preadm_output_df['icustay_id'] == icustayid-200000]['value'].values #preadmission UO ????????????????? Why no + 200000 for icustayid here?
    if len(pread) > 0:     #store the value, if available
        UOtot = np.nansum(pread)
    else:
        UOtot = 0
    #adding the volume of urine produced before start of recording!
    UOnow = np.nansum(output[(output['charttime']>=t0) & (output['charttime'] <= t1)]['value'].values) #t0 and t1 defined above
    UOtot = np.nansum([UOtot, UOnow])


    for j in range(0, 79, timestep): #0:timestep:79 #-28 until +52 = 80 hours in total
        t0 = 3600*j + beg   #left limit of time window
        t1 = 3600*(j + timestep) + beg   #right limit of time window
        ii = (temp[:, 2] >= t0) & (temp[:, 2] <= t1)  #index of items in this time period
        if sum(ii)>0:
            #ICUSTAY_ID, OUTCOMES, DEMOGRAPHICS
            reformat2[irow, 0] = (j/timestep)+1    # 'bloc' = timestep (1,2,3...)
            reformat2[irow, 1] = icustayid         # icustay_ID
            reformat2[irow, 2] = 3600*j+ beg       # t0 = lower limit of time window
            reformat2[irow, 3:11] = dem            # demographics and outcomes

            #CHARTEVENTS and LAB VALUES (+ includes empty cols for shock index and P/F)
            value = temp[ii]  # Records all values in this timestep

            if sum(ii) == 1:   # if only 1 row of values at this timestep
                reformat2[irow, 11:78] = value[:, 3:]
            else:
                reformat2[irow, 11:78] = np.nanmean(value[:, 3:], axis=0)  # mean of all available values

             # MetaVision
            v = ((endv >= t0) & (endv <= t1)) | ((startv >= t0) & (endv<=t1)) | ((startv >= t0) & (startv <= t1))| ((startv <= t0) & (endv>=t1))
            # CareVue
            v2 = vaso2[(vaso2['charttime'] >= t0) & (vaso2['charttime'] <= t1)]['rate_std'].values
            if len(list(ratev[v]) +  list(v2)) > 0:
                v1 = np.nanmedian(list(ratev[v]) +  list(v2))
                v2 = np.nanmax(list(ratev[v]) + list(v2))
            else:
                v1 = np.nan
                v2 = np.nan

            if (~np.isnan(v1)) and (~np.isnan(v2)):
                reformat2[irow, 78] = v1    #median of dose of VP
                reformat2[irow, 79] = v2    #max dose of VP

            # INPUT FLUID
            # Input from MV (4 ways to compute)
            infu = np.nansum(rate*(endt-startt)*((endt <= t1) & (startt >= t0))/3600
                                          + rate*(endt-t0)*((startt <= t0) & (endt <= t1) & (endt >= t0))/3600
                                          + rate*(t1-startt)*((startt >= t0) & (endt >= t1) & (startt <= t1))/3600
                                          + rate*(t1-t0)*((endt >=t1) & (startt <= t0))/3600)
            # All boluses received during this timestep, from inputMV (need to check rate is NaN) and inputCV (simpler):
            bolus = np.nansum(input[(np.isnan(input[:, 5])) & (input[:, 1] >= t0) & (input[:, 1] <= t1), 6]) + np.nansum(input2[(input2[:, 1] >= t0) & (input2[:, 1] <= t1), 4])
            # Cumulate all fluid given
            totvol = np.nansum([totvol, infu, bolus])
            reformat2[irow, 80] = totvol    #total fluid given
            reformat2[irow, 81] = np.nansum([infu, bolus])  #fluid given at this step

            # Urine Output
            UOnow = np.nansum(output[(output['charttime'] >= t0) & (output['charttime'] <= t1)]['value'].values)
            UOtot = np.nansum([UOtot, UOnow])
            reformat2[irow, 82] = UOtot    # Total Urine Output
            reformat2[irow, 83] = np.nansum(UOnow)  # Urine Output at this step

            #CUMULATED BALANCE
            reformat2[irow, 84] = totvol - UOtot

            irow += 1

reformat2 = np.delete(reformat2, range(irow, len(reformat2)) ,axis=0)


Streaming output truncated to the last 5000 lines.
<ipython-input-28-8c411a074f72>:99: RuntimeWarning: Mean of empty slice
  reformat2[irow, 11:78] = np.nanmean(value[:, 3:], axis=0)  # mean of all available values
<ipython-input-28-8c411a074f72>:99: RuntimeWarning: Mean of empty slice
  reformat2[irow, 11:78] = np.nanmean(value[:, 3:], axis=0)  # mean of all available values
<ipython-input-28-8c411a074f72>:99: RuntimeWarning: Mean of empty slice
  reformat2[irow, 11:78] = np.nanmean(value[:, 3:], axis=0)  # mean of all available values
<ipython-input-28-8c411a074f72>:99: RuntimeWarning: Mean of empty slice
  reformat2[irow, 11:78] = np.nanmean(value[:, 3:], axis=0)  # mean of all available values
<ipython-input-28-8c411a074f72>:99: RuntimeWarning: Mean of empty slice
  reformat2[irow, 11:78] = np.nanmean(value[:, 3:], axis=0)  # mean of all available values
<ipython-input-28-8c411a074f72>:99: RuntimeWarning: Mean of empty slice
  reformat2[irow, 11:78] = np.nanmean(value[:, 3:], axis=

In [29]:
# Deleting values with execessive missingness

dataheaders = ['Height_cm', 'Weight_kg', 'GCS','RASS','HR', 'SysBP', 'MeanBP', 'DiaBP',	'RR', 'SpO2', 'Temp_C', 'Temp_F', 'CVP', 'PAPsys', 'PAPmean', 'PAPdia', 'CI',
'SVR', 'Interface', 'FiO2_100', 'FiO2_1', 'O2flow', 'PEEP', 'TidalVolume', 'MinuteVentil', 'PAWmean', 'PAWpeak', 'PAWplateau', 'Potassium', 'Sodium',
'Chloride', 'Glucose', 'BUN', 'Creatinine', 'Magnesium', 'Calcium', 'Ionised_Ca', 'CO2_mEqL', 'SGOT', 'SGPT', 'Total_bili', 'Direct_bili', 'Total_protein',
'Albumin', 'Troponin', 'CRP', 'Hb', 'Ht', 'RBC_count', 'WBC_count', 'Platelets_count', 'PTT', 'PT', 'ACT', 'INR', 'Arterial_pH', 'paO2', 'paCO2',
'Arterial_BE', 'Arterial_lactate', 'HCO3', 'ETCO2', 'SvO2', 'mechvent', 'extubated', 'Shock_Index', 'PaO2_FiO2']
dataheaders = ['bloc','icustayid','charttime','gender','age','elixhauser','re_admission', 'died_in_hosp', 'died_within_48h_of_out_time','mortality_90d','delay_end_of_record_and_discharge_or_death'] + \
    dataheaders + ['median_dose_vaso','max_dose_vaso','input_total','input_4hourly','output_total','output_4hourly','cumulated_balance']

reformat2t = pd.DataFrame(reformat2, columns=dataheaders)
miss = np.sum(np.isnan(reformat2), axis=0)/reformat2.shape[0]

# If values have less than 70% missing values (over 30% of values present): I keep them
reformat3 = reformat2[:,[True]*11 + (miss[11:74] < 0.7).tolist() + [True]*11]
reformat3t = pd.DataFrame(reformat3, columns= reformat2t.columns[[True]*11 + (miss[11:74] < 0.7).tolist() + [True]*11])


In [30]:
# Interpolation

# Do linear interpolation where missingness is low (kNN imputation doesnt work if all rows have missing values)
print('Full ICU -- Doing linear interpolation where missingness is low (kNN imputation doesnt work if all rows have missing values)')
miss = np.sum(np.isnan(reformat3), axis=0)/reformat3.shape[0]
ii = (miss>0) & (miss<0.05)  #less than 5% missingness
mechventcol = reformat3t.columns.tolist().index('mechvent')

for i in range(10,mechventcol): # Correct column by column
    if ii[i]==1:
        reformat3[:,i] = fixgaps(reformat3[:,i])

reformat3t[reformat3t.columns[10:mechventcol]] = reformat3[:,10:mechventcol]

# KNN IMPUTATION -  Done on chunks of 10K records.
print('Full ICU -- KNN imputation')

reformat3t_cols = reformat3t.columns.tolist()
mechventcol = reformat3t_cols.index('mechvent')
ref = np.copy(reformat3[:,11:mechventcol])  #columns of interest

for i in range(0,reformat3.shape[0],9999):   #dataset divided in 10K rows chunks (otherwise too large)
    ref[i:i+9999,:] = KNN(k=1).fit_transform(ref[i:i+9999,:])

reformat3t[reformat3t_cols[11:mechventcol]] = ref

reformat4t = reformat3t.copy()
reformat4 = reformat4t.values

Full ICU -- Doing linear interpolation where missingness is low (kNN imputation doesnt work if all rows have missing values)
Full ICU -- KNN imputation


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Imputing row 1/9999 with 29 missing, elapsed time: 39.874
Imputing row 101/9999 with 3 missing, elapsed time: 40.108
Imputing row 201/9999 with 8 missing, elapsed time: 40.444
Imputing row 301/9999 with 10 missing, elapsed time: 40.762
Imputing row 401/9999 with 27 missing, elapsed time: 41.138
Imputing row 501/9999 with 18 missing, elapsed time: 41.399
Imputing row 601/9999 with 13 missing, elapsed time: 41.588
Imputing row 701/9999 with 1 missing, elapsed time: 41.798
Imputing row 801/9999 with 20 missing, elapsed time: 42.058
Imputing row 901/9999 with 15 missing, elapsed time: 42.255
Imputing row 1001/9999 with 10 missing, elapsed time: 42.458
Imputing row 1101/9999 with 16 missing, elapsed time: 42.659
Imputing row 1201/9999 with 13 missing, elapsed time: 42.865
Imputing row 1301/9999 with 16 missing, elapsed time: 43.119
Imputing row 1401/9999 with 15 missing, elapsed time: 43.331
Imputing row 1501/9999 with 7 missing, elapsed time: 43.524
Imputing row 1601/9999 with 18 missing, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Imputing row 1/9999 with 22 missing, elapsed time: 40.033
Imputing row 101/9999 with 22 missing, elapsed time: 40.152
Imputing row 201/9999 with 6 missing, elapsed time: 40.263
Imputing row 301/9999 with 3 missing, elapsed time: 40.345
Imputing row 401/9999 with 8 missing, elapsed time: 40.448
Imputing row 501/9999 with 13 missing, elapsed time: 40.559
Imputing row 601/9999 with 13 missing, elapsed time: 40.683
Imputing row 701/9999 with 15 missing, elapsed time: 40.820
Imputing row 801/9999 with 18 missing, elapsed time: 40.952
Imputing row 901/9999 with 2 missing, elapsed time: 41.037
Imputing row 1001/9999 with 6 missing, elapsed time: 41.162
Imputing row 1101/9999 with 16 missing, elapsed time: 41.281
Imputing row 1201/9999 with 11 missing, elapsed time: 41.411
Imputing row 1301/9999 with 9 missing, elapsed time: 41.539
Imputing row 1401/9999 with 6 missing, elapsed time: 41.644
Imputing row 1501/9999 with 15 missing, elapsed time: 41.736
Imputing row 1601/9999 with 14 missing, ela

...

Imputing row 1/9999 with 15 missing, elapsed time: 38.434
Imputing row 101/9999 with 24 missing, elapsed time: 38.632
Imputing row 201/9999 with 12 missing, elapsed time: 38.825
Imputing row 301/9999 with 4 missing, elapsed time: 39.014
Imputing row 401/9999 with 10 missing, elapsed time: 39.179
Imputing row 501/9999 with 21 missing, elapsed time: 39.377
Imputing row 601/9999 with 14 missing, elapsed time: 39.581
Imputing row 701/9999 with 6 missing, elapsed time: 39.771
Imputing row 801/9999 with 17 missing, elapsed time: 39.955
Imputing row 901/9999 with 7 missing, elapsed time: 40.121
Imputing row 1001/9999 with 5 missing, elapsed time: 40.290
Imputing row 1101/9999 with 29 missing, elapsed time: 40.475
Imputing row 1201/9999 with 2 missing, elapsed time: 40.626
Imputing row 1301/9999 with 16 missing, elapsed time: 40.818
Imputing row 1401/9999 with 35 missing, elapsed time: 40.974
Imputing row 1501/9999 with 7 missing, elapsed time: 41.142
Imputing row 1601/9999 with 36 missing, el

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Imputing row 1/9999 with 6 missing, elapsed time: 39.872
Imputing row 101/9999 with 18 missing, elapsed time: 39.992
Imputing row 201/9999 with 4 missing, elapsed time: 40.096
Imputing row 301/9999 with 6 missing, elapsed time: 40.216
Imputing row 401/9999 with 6 missing, elapsed time: 40.341
Imputing row 501/9999 with 1 missing, elapsed time: 40.465
Imputing row 601/9999 with 32 missing, elapsed time: 40.580
Imputing row 701/9999 with 13 missing, elapsed time: 40.726
Imputing row 801/9999 with 10 missing, elapsed time: 40.882
Imputing row 901/9999 with 2 missing, elapsed time: 40.991
Imputing row 1001/9999 with 6 missing, elapsed time: 41.092
Imputing row 1101/9999 with 17 missing, elapsed time: 41.194
Imputing row 1201/9999 with 9 missing, elapsed time: 41.328
Imputing row 1301/9999 with 7 missing, elapsed time: 41.460
Imputing row 1401/9999 with 27 missing, elapsed time: 41.582
Imputing row 1501/9999 with 14 missing, elapsed time: 41.702
Imputing row 1601/9999 with 14 missing, elaps

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Imputing row 1/9805 with 2 missing, elapsed time: 39.512
Imputing row 101/9805 with 20 missing, elapsed time: 39.636
Imputing row 201/9805 with 29 missing, elapsed time: 39.759
Imputing row 301/9805 with 14 missing, elapsed time: 39.868
Imputing row 401/9805 with 6 missing, elapsed time: 39.994
Imputing row 501/9805 with 16 missing, elapsed time: 40.121
Imputing row 601/9805 with 11 missing, elapsed time: 40.234
Imputing row 701/9805 with 10 missing, elapsed time: 40.359
Imputing row 801/9805 with 7 missing, elapsed time: 40.480
Imputing row 901/9805 with 16 missing, elapsed time: 40.586
Imputing row 1001/9805 with 16 missing, elapsed time: 40.735
Imputing row 1101/9805 with 22 missing, elapsed time: 40.863
Imputing row 1201/9805 with 13 missing, elapsed time: 40.994
Imputing row 1301/9805 with 10 missing, elapsed time: 41.129
Imputing row 1401/9805 with 12 missing, elapsed time: 41.258
Imputing row 1501/9805 with 10 missing, elapsed time: 41.380
Imputing row 1601/9805 with 12 missing,

In [31]:
# Computing derived variables

# CORRECT GENDER
reformat4t['gender'] = reformat4t['gender'] - 1

# CORRECT AGE > 200 yo
ii = reformat4t['age'] > 150*365.25
reformat4t.loc[ii,'age'] = 91.4*365.25

# FIX MECHVENT
reformat4t['mechvent'].fillna(0, inplace=True)
reformat4t.loc[reformat4t['mechvent'] > 0, 'mechvent'] = 1

# FIX Elixhauser missing values
reformat4t['elixhauser'].loc[np.isnan(reformat4t['elixhauser'])] = np.nanmedian(reformat4t['elixhauser'])   #use the median value / only a few missing data points

# Vasopressors / no NAN
reformat4t['median_dose_vaso'].fillna(0, inplace=True)
reformat4t['max_dose_vaso'].fillna(0, inplace=True)

# Recompute P/F with no missing values...
reformat4t['PaO2_FiO2'] = reformat4t['paO2']/reformat4t['FiO2_1']

# Recompute SHOCK INDEX without NAN and INF
reformat4t['Shock_Index'] = reformat4t['HR']/reformat4t['SysBP']

reformat4t.loc[np.isinf(reformat4t['Shock_Index']), 'Shock_Index'] = np.nan

d = np.nanmean(reformat4t['Shock_Index'])
reformat4t['Shock_Index'].fillna(d, inplace=True)

# SOFA - at each timepoint we need (in this order):
# P/F,  MV,  PLT,  TOT_BILI,  MAP,  NORAD(max),  GCS,  CR,  UO
s = reformat4t[['PaO2_FiO2', 'Platelets_count', 'Total_bili', 'MeanBP', 'max_dose_vaso', 'GCS', 'Creatinine', 'output_4hourly']].values
p = np.arange(5)
s1=np.array([s[:,0]>400, (s[:, 0]>=300) & (s[:, 0]<400), (s[:, 0]>=200) & (s[:, 0]<300), (s[:, 0]>=100) & (s[:, 0]<200), s[:, 0]<100 ])   #count of points for all 6 criteria of sofa
s2=np.array([s[:,1]>150, (s[:, 1]>=100) & (s[:, 1]<150), (s[:, 1]>=50) & (s[:, 1]<100), (s[:, 1]>=20) & (s[:, 1]<50), s[:, 1]<20 ])
s3=np.array([s[:, 2]<1.2, (s[:, 2]>=1.2) & (s[:, 2]<2), (s[:, 2]>=2) & (s[:, 2]<6), (s[:, 2]>=6) & (s[:, 2]<12), s[:, 2]>12 ])
s4=np.array([s[:, 3]>=70, (s[:, 3]<70) & (s[:, 3]>=65), (s[:, 3]<65), (s[:, 4]>0) & (s[:, 4]<=0.1), s[:, 4]>0.1 ])
s5=np.array([s[:, 5]>14, (s[:, 5]>12) & (s[:, 5]<=14), (s[:, 5]>9) & (s[:, 5]<=12), (s[:, 5]>5) & (s[:, 5]<=9), s[:, 5]<=5])
s6=np.array([s[:, 6]<1.2, (s[:, 6]>=1.2) & (s[:, 6]<2), (s[:, 6]>=2) & (s[:, 6]<3.5), ((s[:, 6]>=3.5) & (s[:, 6]<5))|(s[:, 7]<84), (s[:, 6]>5)|(s[:, 7]<34)])

num_columns = reformat4t.shape[1]   #nr of variables in data
newcols_reformat4 = np.zeros((reformat4t.shape[0],7))
for i in range(reformat4t.shape[0]):
    t = max(p[s1[:, i]], default=0) + max(p[s2[:, i]], default=0) + max(p[s3[:, i]], default=0) + max(p[s4[:, i]], default=0) + max(p[s5[:, i]], default=0) + max(p[s6[:, i]], default=0)  #SUM OF ALL 6 CRITERIA
    if t > 0:
        newcols_reformat4[i, :] = [max(p[s1[:, i]], default=0), max(p[s2[:, i]], default=0), max(p[s3[:, i]], default=0), max(p[s4[:, i]], default=0), max(p[s5[:, i]], default=0), max(p[s6[:, i]], default=0), t]

# SIRS - at each timepoint |  need: temp HR RR PaCO2 WBC
s = reformat4t[['Temp_C', 'HR', 'RR', 'paCO2', 'WBC_count']].values

s1=np.array([(s[:, 0]>=38) | (s[:,0]<=36)])   # Count of points for all criteria of SIRS
s2=np.array([s[:, 1]>90])
s3=np.array([(s[:, 2]>=20) | (s[:, 3]<=32)])
s4=np.array([(s[:, 4]>=12) | (s[:, 4]<4)])
newcols_sirs = (1*s1) + (1*s2) + (1*s3) + (1*s4)

# Adds 2 cols for SOFA and SIRS
# Records values
reformat4t['SOFA'] = newcols_reformat4[:,-1]
reformat4t['SIRS'] = newcols_sirs[0]


<ipython-input-31-f6310e1029e6>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reformat4t['mechvent'].fillna(0, inplace=True)
<ipython-input-31-f6310e1029e6>:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are set

In [32]:
# Excluding patients

# Check for patients with extreme UO = outliers = to be deleted (>40 litres of UO per 4h!!)
a = reformat4t['output_4hourly'] > 12000
i = reformat4t[a]['icustayid'].unique()
i = reformat4t['icustayid'].isin(i)
reformat4t.drop(reformat4t.index[i], inplace=True)

# Some have bili = 999999
a = reformat4t['Total_bili'] > 10000
i = reformat4t[a]['icustayid'].unique()
i = reformat4t['icustayid'].isin(i)
reformat4t.drop(reformat4t.index[i], inplace=True)

# Check for patients with extreme INTAKE = outliers = to be deleted (>10 litres of intake per 4h!!)
a = reformat4t['input_4hourly'] > 10000
i = reformat4t[a]['icustayid'].unique()
i = reformat4t['icustayid'].isin(i)
reformat4t.drop(reformat4t.index[i], inplace=True)
########################################################################

# Exclude early deaths from possible withdrawals
print('Full ICU -- Excluding early deaths from possible withdrawals')
# Stats per patient
q = reformat4t['bloc']==1

num_of_trials = len(reformat4t['icustayid'].unique())
a = reformat4t[['icustayid', 'mortality_90d', 'max_dose_vaso', 'SOFA']].values
a = pd.DataFrame(a, columns = ['id', 'mortality_90d', 'vaso', 'sofa'])
d = a.groupby(['id']).max()
d_count = a.groupby(['id']).count()

# Find the patients who match the Sepsis 3 criteria
e = np.zeros(num_of_trials)
for i in range(num_of_trials):
    if d['mortality_90d'].iloc[i] == 1:
        ii = (reformat4t['icustayid'] == d.index[i]) & (reformat4t['bloc'] == d_count.iloc[i]['mortality_90d'])  #last row for this patient
        e[i] = np.sum((reformat4t['max_dose_vaso'][ii] == 0) & (d['vaso'].iloc[i] > 0.3) & (reformat4t['SOFA'][ii] >= d['sofa'].iloc[i]/2)) > 0
r = d.index[(e == 1) & (d_count['mortality_90d'] < 20)] # ids to be removed
ii = reformat4t['icustayid'].isin(r)
reformat4t = reformat4t.loc[~ii]

# Exclude patients who died in ICU during data collection period
print('Full ICU -- excluding patients who died in ICU during data collection period')
ii = (reformat4t['bloc'] == 1) & (reformat4t['died_within_48h_of_out_time'] == 1) & (reformat4t['delay_end_of_record_and_discharge_or_death'] < 24)
ii = reformat4t['icustayid'][ii][reformat4t['icustayid'][ii].isin(icustayidlist)]
ii = reformat4t['icustayid'].isin(ii)
# reformat4t = reformat4t.loc[~ii]

Full ICU -- Excluding early deaths from possible withdrawals
Full ICU -- excluding patients who died in ICU during data collection period


In [33]:
# Sepsis Cohort from all ICU Patients extracted

sepsis = np.zeros((30000,5))
irow = 0

for icustayid in icustayidlist:
    ii = reformat4t['icustayid'] == icustayid
    if sum(ii) > 0:
        sofa = reformat4t['SOFA'][ii]
        sirs = reformat4t['SIRS'][ii]
        sepsis[irow, 0] = icustayid
        sepsis[irow, 1] = reformat4t['mortality_90d'][ii].iloc[0] # 90-day mortality
        sepsis[irow, 2] = np.max(sofa)
        sepsis[irow, 3] = np.max(sirs)
        sepsis[irow, 4] = qstime[icustayid][0]   #time of onset of sepsis #icustayid-1 not done to keep it consistent with earlier verified use of qstime and 0 added as onset of sepsis index.
        irow += 1

sepsis = np.delete(sepsis, range(irow, len(sepsis)) ,axis=0) # Remove extra rows
sepsis = pd.DataFrame(sepsis, columns=['icustayid', 'morta_90d', 'max_sofa', 'max_sirs', 'sepsis_time'])

# Delete all non-septic patients
ii = sepsis['max_sofa'] < 2
sepsis = sepsis[~ii]
# Final count of patients included
print('Final patient count:', sepsis.shape[0])

# Save cohort
if pargs.save_intermediate:
    sepsis.to_csv(project_path/'new_sepsis_mimiciii.csv', index=False)

Final patient count: 17635


In [34]:
# Other sepsis cohort

reformat = np.nan*np.ones((2000000,69))  #final table
qstime = dict()
winb4 = 25   #lower limit for inclusion of data (24h before time flag)
winaft = 49  # upper limit (48h after)
irow = 0  #recording row for summary table

for icustayidrow in range(1, sepsis.shape[0]+1):
    qst = sepsis['sepsis_time'].iloc[icustayidrow-1] #;%,3); %flag for presumed infection
    icustayid = int(sepsis['icustayid'].iloc[icustayidrow-1])
    qstime[icustayid] = np.zeros(4)
    # CHARTEVENTS
    if (icustayid-200000) < 10000:
        temp=ce010
    elif (icustayid-200000) < 20000:
        temp=ce1020
    elif (icustayid-200000) < 30000:
        temp=ce2030
    elif (icustayid-200000) < 40000:
        temp=ce3040
    elif (icustayid-200000) < 50000:
        temp=ce4050
    elif (icustayid-200000) < 60000:
        temp=ce5060
    elif (icustayid-200000) < 70000:
        temp=ce6070
    elif (icustayid-200000) < 80000:
        temp=ce7080
    elif (icustayid-200000) < 90000:
        temp=ce8090
    else:
        temp=ce90100
    temp = temp[temp['icustay_id'] == icustayid]

    ii = (temp['charttime'] >= qst - (winb4+4)*3600) & (temp['charttime'] <= qst + (winaft+4)*3600) #time period of interest -4h and +4h
    temp = temp.loc[ii]   #only time period of interest

    # LAB EVENTS
    ii = labU['icustay_id'] == icustayid
    temp2 = labU.loc[ii]
    ii = (temp2['charttime'] >= qst - (winb4+4)*3600) & (temp2['charttime'] <= qst + (winaft+4)*3600) #time period of interest -4h and +4h
    temp2 = temp2.loc[ii]   #only time period of interest

    # Mech Vent + ?extubated
    ii = mechvent_df['icustay_id'] == icustayid
    temp3 = mechvent_df.loc[ii]
    ii = (temp3['charttime'] >= qst - (winb4+4)*3600) & (temp3['charttime'] <= qst + (winaft+4)*3600) # Time period of interest -4h and +4h
    temp3 = temp3.loc[ii]   #only time period of interest

    t = np.unique(pd.concat([temp['charttime'], temp2['charttime'], temp3['charttime']], ignore_index=True).values) # List of unique timestamps from all 3 sources / sorted in ascending order

    if len(t) > 0:
        for i in range(len(t)):
            # CHARTEVENTS
            ii = temp['charttime'] == t[i]
            col = temp.loc[ii, 'itemid']
            value = temp.loc[ii, 'valuenum']
            reformat[irow, 0] = i+1 # Timestep
            reformat[irow, 1] = icustayid
            reformat[irow, 2] = t[i] # Charttime
            reformat[irow, 3] = qst  # Store the presumed onset time
            reformat[irow, 3+col.astype(int).values] = value.values # Store available values

            #LAB VALUES
            ii = temp2['charttime'] == t[i]
            col = temp2.loc[ii, 'itemid']
            value = temp2.loc[ii, 'valuenum']
            reformat[irow,31+col.astype(int).values] = value.values  #store available values

            #MV
            ii = temp3['charttime'] == t[i]
            if np.nansum(ii) > 0:
                col = temp3.loc[ii, 'MechVent']
                value = temp3.loc[ii, 'Extubated']
                reformat[irow, 67] = col.values[0] # Store available values
                reformat[irow, 68] = value.values[0] # Store available values
            else:
                reformat[irow, 67]= np.nan
                reformat[irow, 68]= np.nan
            irow += 1

        qstime[icustayid][0] = qst # Flag for presumed infection / this is time of sepsis if SOFA >=2 for this patient
        # WE SAVE FIRST and LAST TIMESTAMPS, in QSTIME, for each ICUSTAYID
        qstime[icustayid][1] = t[0]   # First timestamp
        qstime[icustayid][2] = t[-1]  # Last timestamp
        qstime[icustayid][3] = demographics_df.loc[demographics_df['icustay_id'] == icustayid, 'dischtime'].values[0] # Discharge time

reformat = np.delete(reformat, range(irow, len(reformat)) ,axis=0)  # Remove unused rows


In [35]:
# Outliers

# Weight
reformat = deloutabove(reformat, 5, 300)

# Heart Rate
reformat = deloutabove(reformat, 8, 250)

# Blood Pressure
reformat = deloutabove(reformat, 9, 300)
reformat = deloutbelow(reformat, 10, 0)
reformat = deloutabove(reformat, 11, 200)
reformat = deloutbelow(reformat, 11, 0)
reformat = deloutabove(reformat, 11, 200)

# Respiratory Rate
reformat = deloutabove(reformat, 12, 80)

# SpO2
reformat = deloutabove(reformat, 13, 150)
reformat[reformat[:, 13]>100, 13] = 100
reformat = deloutbelow(reformat, 13, 50)

# Temperature
reformat[(reformat[:, 14] > 90) & (np.isnan(reformat[:, 15])), 15] = reformat[(reformat[:, 14] > 90) & (np.isnan(reformat[:, 15])), 14]
reformat = deloutabove(reformat, 14, 90)
reformat = deloutbelow(reformat, 14, 25)

# interface / is in col 23
# FiO2
reformat = deloutabove(reformat, 23, 100)
reformat[reformat[:, 23] < 1 , 23] = reformat[reformat[:, 23] < 1 , 23]*100
reformat = deloutbelow(reformat, 23, 20)
reformat = deloutabove(reformat, 24, 1.5)

# O2 FLOW
reformat = deloutabove(reformat, 25, 70)

# PEEP
reformat=deloutbelow(reformat, 26, 0)
reformat=deloutabove(reformat, 26, 40)

# Total Volume
reformat=deloutabove(reformat, 27, 1800)

# Mean Volume
reformat=deloutabove(reformat, 28, 50)

# Potassium
reformat=deloutbelow(reformat, 32, 1)
reformat=deloutabove(reformat, 32, 15)

# Sodium
reformat=deloutbelow(reformat, 33, 95)
reformat=deloutabove(reformat, 33, 178)

# Chloride
reformat=deloutbelow(reformat, 34, 70)
reformat=deloutabove(reformat, 34, 150)

# Glucose
reformat=deloutbelow(reformat, 35, 1)
reformat=deloutabove(reformat, 35, 1000)

# Creatinine
reformat=deloutabove(reformat, 37, 150)

# Magnesium
reformat=deloutabove(reformat, 38, 10)

# Calcium
reformat=deloutabove(reformat, 39, 20)

# Ionized Calcium
reformat=deloutabove(reformat, 40, 5)

# CO2
reformat=deloutabove(reformat, 41, 120)

# SGPT/SGOT
reformat=deloutabove(reformat, 42, 10000)
reformat=deloutabove(reformat, 43, 10000)

# Hb/Ht
reformat=deloutabove(reformat, 50, 20)
reformat=deloutabove(reformat, 51, 65)

# White Blood Cells
reformat=deloutabove(reformat, 53, 500)

# Platelets
reformat=deloutabove(reformat, 54, 2000)

# INR
reformat=deloutabove(reformat, 58, 20)

# pH
reformat=deloutbelow(reformat, 59, 6.7)
reformat=deloutabove(reformat, 59, 8)

# pO2
reformat=deloutabove(reformat, 60, 700)

# pCO2
reformat=deloutabove(reformat, 61, 200)

# Base Excess
reformat=deloutbelow(reformat, 62, -50)

# Lactate
reformat=deloutabove(reformat, 63, 30)

In [36]:
# Estimate GCS from RASS - data from Wesley JAMA 2003
reformat[(np.isnan(reformat[:, 6])) & (reformat[:, 7] >= 0), 6] = 15
reformat[(np.isnan(reformat[:, 6])) & (reformat[:, 7] == -1), 6] = 14
reformat[(np.isnan(reformat[:, 6])) & (reformat[:, 7] == -2), 6] = 12
reformat[(np.isnan(reformat[:, 6])) & (reformat[:, 7] == -3), 6] = 11
reformat[(np.isnan(reformat[:, 6])) & (reformat[:, 7] == -4), 6] = 6
reformat[(np.isnan(reformat[:, 6])) & (reformat[:, 7] == -5), 6] = 3

# FiO2
reformat[(~np.isnan(reformat[:, 23])) & (np.isnan(reformat[:, 24])), 24] = reformat[(~np.isnan(reformat[:, 23])) & (np.isnan(reformat[:, 24])), 23] / 100
reformat[(~np.isnan(reformat[:, 24])) & (np.isnan(reformat[:, 23])), 23] = reformat[(~np.isnan(reformat[:, 24])) & (np.isnan(reformat[:, 23])), 24] * 100

# ESTIMATE FiO2 /// with use of interface / device (cannula, mask, ventilator....)
print('Sepsis Cohort -- Doing sample and hold')
reformatsah = SAH(reformat, sample_and_hold, adjust=1) # do SAH first to handle this task, setting `adjust` to 1 since we added another column

# NO FiO2, YES O2 flow, no interface OR cannula
ii = np.where((np.isnan(reformatsah[:,23])) & (~np.isnan(reformatsah[:,25])) & ((reformatsah[:,22] == 0) | (reformatsah[:,22] == 2)))[0] #As np.where returns a tuple
reformat[ii[reformatsah[ii, 25] <= 15], 23] = 70
reformat[ii[reformatsah[ii, 25] <= 12], 23] = 62
reformat[ii[reformatsah[ii, 25] <= 10], 23] = 55
reformat[ii[reformatsah[ii, 25] <= 8], 23]  = 50
reformat[ii[reformatsah[ii, 25] <= 6], 23]  = 44
reformat[ii[reformatsah[ii, 25] <= 5], 23]  = 40
reformat[ii[reformatsah[ii, 25] <= 4], 23]  = 36
reformat[ii[reformatsah[ii, 25] <= 3], 23]  = 32
reformat[ii[reformatsah[ii, 25] <= 2], 23]  = 28
reformat[ii[reformatsah[ii, 25] <= 1], 23]  = 24

# NO FiO2, NO O2 flow, no interface OR cannula
ii = np.where((np.isnan(reformatsah[:, 23])) & np.isnan(reformatsah[:, 25]) & ((reformatsah[:, 22] == 0) | (reformatsah[:, 22] == 2)))[0]  #no fio2 given and o2flow given, no interface OR cannula
reformat[ii, 23] = 21

# NO FiO2, YES O2 flow, face mask OR.... OR ventilator (assume it's face mask)
ii = np.where((np.isnan(reformatsah[:,23])) & (~np.isnan(reformatsah[:,25])) &
((reformatsah[:, 22]==1) | (reformatsah[:, 22]==3) | (reformatsah[:, 22]==4) | (reformatsah[:, 22]==5) | (reformatsah[:, 22]==6) | (reformatsah[:, 22]==9) | (reformatsah[:, 22]==10)))[0]
reformat[ii[reformatsah[ii, 25]<=15], 23] = 75
reformat[ii[reformatsah[ii, 25]<=12], 23] = 69
reformat[ii[reformatsah[ii, 25]<=10], 23] = 66
reformat[ii[reformatsah[ii, 25]<=8], 23]  = 58
reformat[ii[reformatsah[ii, 25]<=6], 23]  = 40
reformat[ii[reformatsah[ii, 25]<=4], 23]  = 36

# NO FiO2, NO O2 flow, face mask OR ....OR ventilator
ii = np.where(np.isnan(reformatsah[:, 23]) & np.isnan(reformatsah[:, 25]) & ((reformatsah[:, 22] == 1) | (reformatsah[:, 22] == 3) |
(reformatsah[:, 22] == 4) | (reformatsah[:, 22] == 5) | (reformatsah[:, 22] == 6) | (reformatsah[:, 22] == 9) | (reformatsah[:, 22] == 10)))[0]  # No FiO2 given and O2 flow given, no interface OR cannula
reformat[ii, 23] = np.nan

# NO FiO2, YES O2 flow, Non rebreather mask
ii = np.where(np.isnan(reformatsah[:, 23]) & (~np.isnan(reformatsah[:, 25])) & (reformatsah[:, 22] == 7))[0]
reformat[ii[reformatsah[ii, 25] >= 10], 23] = 90
reformat[ii[reformatsah[ii, 25] >= 15], 23] = 100
reformat[ii[reformatsah[ii, 25] < 10], 23]  = 80
reformat[ii[reformatsah[ii, 25] <= 8], 23]  = 70
reformat[ii[reformatsah[ii, 25] <= 6], 23]  = 60

# NO FiO2, NO O2 flow, NRM
ii= np.where(np.isnan(reformatsah[:, 23]) & np.isnan(reformatsah[:, 25]) & (reformatsah[:, 22]==7))[0]  #no fio2 given and o2flow given, no interface OR cannula
reformat[ii, 23] = np.nan

# Update Fi02 columns again
ii = (~np.isnan(reformat[:, 23])) & (np.isnan(reformat[:, 24]))
reformat[ii, 24] = reformat[ii, 23]/100
ii = (~np.isnan(reformat[:, 24])) & (np.isnan(reformat[:, 24]))
reformat[ii, 23] = reformat[ii, 24]*100

# BLOOD PRESSURE
ii = (~np.isnan(reformat[:, 9])) & (~np.isnan(reformat[:, 10])) & np.isnan(reformat[:, 11])
reformat[ii, 11] = (3*reformat[ii, 10] - reformat[ii, 9])/2
ii = (~np.isnan(reformat[:, 9])) & (~np.isnan(reformat[:, 11])) & np.isnan(reformat[:, 10])
reformat[ii, 10] = (reformat[ii, 9] + 2*reformat[ii, 11])/3
ii = (~np.isnan(reformat[:, 10])) & (~np.isnan(reformat[:, 11])) & np.isnan(reformat[:, 9])
reformat[ii, 9] = 3*reformat[ii, 10] - 2*reformat[ii, 11]

# TEMPERATURE
# some values recorded in the wrong column
ii = (reformat[:, 15] > 25) & (reformat[:, 15] < 45) #tempF close to 37deg??!
reformat[ii, 14] = reformat[ii, 15]
reformat[ii, 15] = np.nan
ii = reformat[:, 14] >70  # TempC > 70, likely recorded in Farenheit
reformat[ii, 15] = reformat[ii, 14]
reformat[ii, 14] = np.nan
ii = (~np.isnan(reformat[:, 14])) & np.isnan(reformat[:, 15])
reformat[ii, 15] = reformat[ii, 14] * 1.8 + 32
ii = (~np.isnan(reformat[:, 14])) & np.isnan(reformat[:, 13])
reformat[ii, 14] = (reformat[ii, 15] - 32)/1.8

# Hb/Ht
ii = (~np.isnan(reformat[:, 50])) & np.isnan(reformat[:,51])
reformat[ii, 51] = (reformat[ii, 50] * 2.862) + 1.216
ii = (~np.isnan(reformat[:, 51])) & np.isnan(reformat[:,50])
reformat[ii, 50] = (reformat[ii, 51] - 1.216)/2.862

# BILIRUBIN
ii = (~np.isnan(reformat[:, 44])) & np.isnan(reformat[:,45])
reformat[ii, 45] = (reformat[ii, 44]*0.6934)-0.1752
ii = (~np.isnan(reformat[:, 45])) & np.isnan(reformat[:,44])
reformat[ii, 44] = (reformat[ii, 45] + 0.1752)/0.6934

Sepsis Cohort -- Doing sample and hold


In [37]:
# Sample and hold raw data

reformat = SAH(reformat[:, 0:69], sample_and_hold, adjust=1)  # Setting `adjust` to 1 to account for the added column for `presumed_onset`

In [38]:
# Data combination

timestep = 4  # Resolution of timesteps, in hours
irow = 0
icustayidlist = np.unique(reformat[:,1]).astype(np.int32)
reformat2 = np.nan*np.ones((reformat.shape[0], 86))  # Output array
num_patients = len(icustayidlist)  # Number of patients

# Adding 2 empty cols for future shock index=HR/SBP and P/F
reformat = np.hstack([reformat, np.nan*np.ones((reformat.shape[0], 2))])

for i in range(len(icustayidlist)):
    icustayid = icustayidlist[i]

    #CHARTEVENTS AND LAB VALUES
    temp = reformat[reformat[:,1] == icustayid,:]   # Subtable of interest
    beg = temp[0,2]   # Timestamp of first record

    #IV FLUID STUFF
    iv = mv_fluids_df['icustay_id'] == icustayid   # Rows of interest in inputMV
    input = mv_fluids_df[iv]    # Subset of interest
    iv = cv_fluids_df['icustay_id'] == icustayid   # Rows of interest in inputCV
    input2 = cv_fluids_df[iv]    # Subset of interest
    startt = input['starttime'] # Start of all infusions and boluses
    endt = input['endtime'] # End of all infusions and boluses
    rate = input['norm_rate_of_infusion']  # Rate of infusion (is NaN for boluses) || corrected for tonicity
    pread = preadm_fluids_df[preadm_fluids_df['icustay_id'] == icustayid]['inputpreadm'] # Preadmission volume

    input = input.values
    input2 = input2.values

    if len(pread) >0:             # Store the value, if available
        totvol = np.nansum(pread)
    else:
        totvol=0   # If not documented: it's zero

    # Compute volume of fluid given before start of record!!!
    t0 = 0
    t1 = beg
    # Input from MV (4 ways to compute)
    infu = np.nansum(rate*(endt-startt)*((endt<=t1) & (startt >= t0))/3600 + rate*(endt-t0)*((startt <= t0) & (endt <= t1) & (endt >= t0))/3600 +
                            rate*(t1-startt)*((startt >= t0) & (endt >=t1) & (startt <= t1))/3600 + rate*(t1-t0)*((endt >= t1) & (startt <= t0))/3600)
    # All boluses received during this timestep, from inputMV (need to check rate is NaN) and inputCV (simpler):
    bolus = np.nansum(input[np.isnan(input[:,5]) & (input[:,1] >= t0) & (input[:,1] <= t1),6]) + np.nansum(input2[(input2[:,1] >= t0) & (input2[:,1] <= t1),4])
    totvol = np.nansum([totvol,infu,bolus])

    #########################################################################################
    #VASOPRESSORS
    iv = mv_vaso_df['icustay_id'] == icustayid # rows of interest in vasoMV
    vaso1 = mv_vaso_df[iv]    # subset of interest
    iv = cv_vasp_df['icustay_id'] == icustayid   # rows of interest in vasoCV
    vaso2 = cv_vasp_df[iv]    # subset of interest
    startv = vaso1['starttime'].values  # start of VP infusion
    endv = vaso1['endtime'].values      # end of VP infusions
    ratev = vaso1['rate_std'].values  # rate of VP infusion

    # DEMOGRAPHICS / gender, age, elixhauser, re-admit, onset time, died in hosp?, died within 48h of out_time (likely in ICU or soon after),
    # died within 90d after admission?, Length of stay after obs window
    demogi = demographics_df['icustay_id'] == icustayid
    dem = np.array(
            list(demographics_df.gender[demogi].values) +
            list(demographics_df.age[demogi].values) +
            list(demographics_df.elixhauser[demogi].values) +
            list(demographics_df.re_admission[demogi].values) + # Using corrected readmission (within 30 days of previous discharge)
            [temp[0,3]] +                    # Adding presumed sepsis onset
            list(demographics_df.morta_hosp[demogi].values) +
            list(abs(demographics_df.dod[demogi].values - demographics_df.outtime[demogi].values) < (24*3600*2)) +
            list(demographics_df.morta_90[demogi].values) +
            [(qstime[icustayid][3] - qstime[icustayid][2])/3600]      # Length of time after last observation and discharge?
            )

    # URINE OUTPUT
    iu = output_df['icustay_id'] == icustayid   #rows of interest in inputMV
    output = output_df[iu]    #subset of interest
    pread = preadm_output_df[preadm_output_df['icustay_id'] == icustayid-200000]['value'].values #preadmission UO ????????????????? Why no + 200000 for icustayid here?
    if len(pread) > 0:     #store the value, if available
        UOtot = np.nansum(pread)
    else:
        UOtot = 0
    # adding the volume of urine produced before start of recording!
    UOnow = np.nansum(output[(output['charttime']>=t0) & (output['charttime'] <= t1)]['value'].values) #t0 and t1 defined above
    UOtot = np.nansum([UOtot, UOnow])

    # Loop over the relevant times for each patient where information is recorded
    for j in range(0, 79, timestep): # 0:timestep:79 % -28 until +52 = 80 hours in total (4 hours buffer on either side of our desired window)
        t0 = 3600*j + beg   # left limit of time window
        t1 = 3600*(j + timestep) + beg   # right limit of time window
        ii = (temp[:,2] >= t0) & (temp[:,2] <= t1)  # index of items in this time period
        if sum(ii)>0:
            # ICUSTAY_ID, OUTCOMES, DEMOGRAPHICS
            reformat2[irow,0] = (j/timestep)+1    # 'bloc' = timestep (1,2,3...)
            reformat2[irow,1] = icustayid         # icustay_ID
            reformat2[irow,2] = 3600*j+ beg       # t0 = lower limit of time window
            reformat2[irow,3:12] = dem            # demographics and outcomes

            # CHARTEVENTS and LAB VALUES (+ includes empty columns for shock index and P/F)
            value = temp[ii]  #records all values in this timestep

            if sum(ii) == 1:   # if only 1 row of values at this timestep
                reformat2[irow,12:79] = value[:,4:]
            else:
                reformat2[irow,12:79] = np.nanmean(value[:,4:], axis=0) # mean of all available values

            #MV
            v = ((endv >= t0) & (endv <= t1)) | ((startv >= t0) & (endv<=t1)) | ((startv >= t0) & (startv <= t1))| ((startv <= t0) & (endv>=t1))
            #CV
            v2 = vaso2[(vaso2['charttime'] >= t0) & (vaso2['charttime'] <= t1)]['rate_std'].values
            if len(list(ratev[v]) +  list(v2)) > 0:
                v1 = np.nanmedian(list(ratev[v]) +  list(v2))
                v2 = np.nanmax(list(ratev[v]) + list(v2))
            else:
                v1 = np.nan
                v2 = np.nan
            if (~np.isnan(v1)) and (~np.isnan(v2)):
                reformat2[irow,79] = v1    #median of dose of VP
                reformat2[irow,80] = v2    #max dose of VP

            #INPUT FLUID
            #input from MV (4 ways to compute)
            infu = np.nansum(rate*(endt-startt)*((endt <= t1) & (startt >= t0))/3600
                                          + rate*(endt-t0)*((startt <= t0) & (endt <= t1) & (endt >= t0))/3600
                                          + rate*(t1-startt)*((startt >= t0) & (endt >= t1) & (startt <= t1))/3600
                                          + rate*(t1-t0)*((endt >=t1) & (startt <= t0))/3600)
            #all boluses received during this timestep, from inputMV (need to check rate is NaN) and inputCV (simpler):
            bolus = np.nansum(input[(np.isnan(input[:,5])) & (input[:,1] >= t0) & (input[:,1] <= t1),6]) + np.nansum(input2[(input2[:,1] >= t0) & (input2[:,1] <= t1),4])
            #sum fluid given
            totvol = np.nansum([totvol, infu, bolus])
            reformat2[irow,81] = totvol    #total fluid given
            reformat2[irow,82] = np.nansum([infu, bolus])  #fluid given at this step

            #UO
            UOnow = np.nansum(output[(output['charttime'] >= t0) & (output['charttime'] <= t1)]['value'].values)
            UOtot = np.nansum([UOtot, UOnow])
            reformat2[irow,83] = UOtot    #total UO
            reformat2[irow,84] = np.nansum(UOnow)   #UO at this step

            #CUMULATED BALANCE
            reformat2[irow,85] = totvol - UOtot    #cumulated balance

            irow += 1

reformat2 = np.delete(reformat2, range(irow, len(reformat2)) ,axis=0)



Streaming output truncated to the last 5000 lines.
<ipython-input-38-e3e09d1d4399>:102: RuntimeWarning: Mean of empty slice
  reformat2[irow,12:79] = np.nanmean(value[:,4:], axis=0) # mean of all available values
<ipython-input-38-e3e09d1d4399>:102: RuntimeWarning: Mean of empty slice
  reformat2[irow,12:79] = np.nanmean(value[:,4:], axis=0) # mean of all available values
<ipython-input-38-e3e09d1d4399>:102: RuntimeWarning: Mean of empty slice
  reformat2[irow,12:79] = np.nanmean(value[:,4:], axis=0) # mean of all available values
<ipython-input-38-e3e09d1d4399>:102: RuntimeWarning: Mean of empty slice
  reformat2[irow,12:79] = np.nanmean(value[:,4:], axis=0) # mean of all available values
<ipython-input-38-e3e09d1d4399>:102: RuntimeWarning: Mean of empty slice
  reformat2[irow,12:79] = np.nanmean(value[:,4:], axis=0) # mean of all available values
<ipython-input-38-e3e09d1d4399>:102: RuntimeWarning: Mean of empty slice
  reformat2[irow,12:79] = np.nanmean(value[:,4:], axis=0) # mean o

In [39]:
# Convert table and only keep wanted variables

dataheaders = [i[1:-1] for i in sample_and_hold.columns]
dataheaders.extend(['Shock_Index', 'PaO2_FiO2'])
dataheaders = ['bloc', 'icustayid', 'charttime', 'gender', 'age', 'elixhauser', 're_admission', 'presumed_onset', 'died_in_hosp', \
    'died_within_48h_of_out_time', 'mortality_90d', 'delay_end_of_record_and_discharge_or_death'] + \
    dataheaders + ['median_dose_vaso','max_dose_vaso','input_total','input_4hourly','output_total','output_4hourly','cumulated_balance']


reformat2t = pd.DataFrame(reformat2, columns=dataheaders)

# headers I want to keep
dataheaders5 = ['bloc', 'icustayid', 'charttime', 'gender', 'age', 'elixhauser', 're_admission', 'presumed_onset', 'died_in_hosp', 'died_within_48h_of_out_time', \
    'mortality_90d', 'delay_end_of_record_and_discharge_or_death', 'Weight_kg', 'GCS', 'HR', 'SysBP', 'MeanBP', 'DiaBP', 'RR', 'SpO2', 'Temp_C', 'FiO2_1', \
    'Potassium', 'Sodium', 'Chloride', 'Glucose', 'BUN', 'Creatinine', 'Magnesium', 'Calcium', 'Ionised_Ca', 'CO2_mEqL', 'SGOT', 'SGPT', 'Total_bili', 'Albumin', \
    'Hb', 'WBC_count', 'Platelets_count', 'PTT', 'PT', 'INR', 'Arterial_pH', 'paO2', 'paCO2', 'Arterial_BE', 'HCO3', 'Arterial_lactate', 'mechvent', 'Shock_Index', \
    'PaO2_FiO2', 'median_dose_vaso', 'max_dose_vaso', 'input_total', 'input_4hourly', 'output_total', 'output_4hourly', 'cumulated_balance']

reformat3t = reformat2t[dataheaders5].copy()

## SOME DATA MANIPULATION BEFORE IMPUTATION
#CORRECT GENDER
reformat3t['gender'] = reformat3t['gender'] - 1

# CORRECT AGE > 200 yo
ii = reformat3t['age'] > 150*365.25
reformat3t.loc[ii,'age'] = 91.4*365.25

# FIX MECHVENT
reformat3t['mechvent'].fillna(0, inplace=True)
reformat3t.loc[reformat3t['mechvent'] > 0, 'mechvent'] = 1

# FIX Elixhauser missing values
reformat3t['elixhauser'].loc[np.isnan(reformat3t['elixhauser'])] = np.nanmedian(reformat3t['elixhauser'])   #use the median value / only a few missing data points

# Vasopressors / no NAN
reformat3t['median_dose_vaso'].fillna(0, inplace=True)
reformat3t['max_dose_vaso'].fillna(0, inplace=True)

# Check missingness proportions here
miss = pd.DataFrame([np.sum(np.isnan(reformat3t.values), axis=0)/reformat3t.shape[0]], columns=reformat3t.columns)
# Fill the values temporarily with zeros, otherwise kNN imp doesnt work
reformat3t['Shock_Index'] = np.zeros(reformat3t.shape[0])
reformat3t['PaO2_FiO2'] = np.zeros(reformat3t.shape[0])

<ipython-input-39-c17507aad7e4>:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reformat3t['mechvent'].fillna(0, inplace=True)
<ipython-input-39-c17507aad7e4>:34: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are set

In [40]:
# Handling missing values
reformat3 = reformat3t.values
miss = np.sum(np.isnan(reformat3), axis=0)/reformat3.shape[0]
ii = (miss>0) & (miss<0.05)  #less than 5% missingness
mechventcol = reformat3t.columns.tolist().index('mechvent')

for i in range(12, mechventcol): # correct col by col, otherwise it does it wrongly
    if ii[i]==1:
        reformat3[:, i] = fixgaps(reformat3[:, i])

reformat3t[reformat3t.columns[12:mechventcol]] = reformat3[:, 12:mechventcol]

# KNN IMPUTATION - Done on chunks of 10K records.
print('Sepsis Cohort -- KNN imputation with K = 1')

reformat3t_cols = reformat3t.columns.tolist()
mechventcol = reformat3t_cols.index('mechvent')
ref = np.copy(reformat3[:,12:mechventcol])  #columns of interest


for i in range(0,reformat3.shape[0],9999):   # Dataset divided in 10K rows chunks (otherwise too large)
    ref[i:i+9999,:] = KNN(k=1).fit_transform(ref[i:i+9999,:])

# Copy on the interpolated data
reformat3t[reformat3t_cols[12:mechventcol]] = ref

reformat4t = reformat3t.copy()  # Make final table copy to compute derived features



Sepsis Cohort -- KNN imputation with K = 1


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Imputing row 1/9999 with 22 missing, elapsed time: 35.912
Imputing row 101/9999 with 4 missing, elapsed time: 36.006
Imputing row 201/9999 with 8 missing, elapsed time: 36.103
Imputing row 301/9999 with 11 missing, elapsed time: 36.212
Imputing row 401/9999 with 22 missing, elapsed time: 36.338
Imputing row 501/9999 with 12 missing, elapsed time: 36.461
Imputing row 601/9999 with 10 missing, elapsed time: 36.546
Imputing row 701/9999 with 3 missing, elapsed time: 36.641
Imputing row 801/9999 with 17 missing, elapsed time: 36.791
Imputing row 901/9999 with 12 missing, elapsed time: 36.937
Imputing row 1001/9999 with 8 missing, elapsed time: 37.084
Imputing row 1101/9999 with 16 missing, elapsed time: 37.228
Imputing row 1201/9999 with 7 missing, elapsed time: 37.363
Imputing row 1301/9999 with 20 missing, elapsed time: 37.513
Imputing row 1401/9999 with 9 missing, elapsed time: 37.666
Imputing row 1501/9999 with 9 missing, elapsed time: 37.791
Imputing row 1601/9999 with 16 missing, ela

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Imputing row 1/9999 with 11 missing, elapsed time: 37.972
Imputing row 101/9999 with 6 missing, elapsed time: 38.066
Imputing row 201/9999 with 6 missing, elapsed time: 38.165
Imputing row 301/9999 with 25 missing, elapsed time: 38.243
Imputing row 401/9999 with 18 missing, elapsed time: 38.346
Imputing row 501/9999 with 16 missing, elapsed time: 38.448
Imputing row 601/9999 with 18 missing, elapsed time: 38.547
Imputing row 701/9999 with 10 missing, elapsed time: 38.641
Imputing row 801/9999 with 19 missing, elapsed time: 38.741
Imputing row 901/9999 with 14 missing, elapsed time: 38.814
Imputing row 1001/9999 with 6 missing, elapsed time: 38.907
Imputing row 1101/9999 with 5 missing, elapsed time: 39.007
Imputing row 1201/9999 with 14 missing, elapsed time: 39.114
Imputing row 1301/9999 with 17 missing, elapsed time: 39.225
Imputing row 1401/9999 with 16 missing, elapsed time: 39.302
Imputing row 1501/9999 with 3 missing, elapsed time: 39.388
Imputing row 1601/9999 with 11 missing, e

...

Imputing row 1/9999 with 9 missing, elapsed time: 36.310
Imputing row 101/9999 with 14 missing, elapsed time: 36.449
Imputing row 201/9999 with 3 missing, elapsed time: 36.612
Imputing row 301/9999 with 11 missing, elapsed time: 36.737
Imputing row 401/9999 with 6 missing, elapsed time: 36.861
Imputing row 501/9999 with 24 missing, elapsed time: 37.015
Imputing row 601/9999 with 5 missing, elapsed time: 37.154
Imputing row 701/9999 with 10 missing, elapsed time: 37.279
Imputing row 801/9999 with 27 missing, elapsed time: 37.446
Imputing row 901/9999 with 25 missing, elapsed time: 37.630
Imputing row 1001/9999 with 9 missing, elapsed time: 37.757
Imputing row 1101/9999 with 22 missing, elapsed time: 37.918
Imputing row 1201/9999 with 15 missing, elapsed time: 38.065
Imputing row 1301/9999 with 8 missing, elapsed time: 38.215
Imputing row 1401/9999 with 18 missing, elapsed time: 38.393
Imputing row 1501/9999 with 3 missing, elapsed time: 38.500
Imputing row 1601/9999 with 4 missing, elap

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Imputing row 1/9999 with 16 missing, elapsed time: 37.553
Imputing row 101/9999 with 12 missing, elapsed time: 37.647
Imputing row 201/9999 with 12 missing, elapsed time: 37.772
Imputing row 301/9999 with 6 missing, elapsed time: 37.873
Imputing row 401/9999 with 3 missing, elapsed time: 37.977
Imputing row 501/9999 with 6 missing, elapsed time: 38.062
Imputing row 601/9999 with 10 missing, elapsed time: 38.148
Imputing row 701/9999 with 26 missing, elapsed time: 38.237
Imputing row 801/9999 with 7 missing, elapsed time: 38.335
Imputing row 901/9999 with 18 missing, elapsed time: 38.422
Imputing row 1001/9999 with 13 missing, elapsed time: 38.523
Imputing row 1101/9999 with 13 missing, elapsed time: 38.616
Imputing row 1201/9999 with 17 missing, elapsed time: 38.719
Imputing row 1301/9999 with 5 missing, elapsed time: 38.852
Imputing row 1401/9999 with 5 missing, elapsed time: 38.971
Imputing row 1501/9999 with 9 missing, elapsed time: 39.052
Imputing row 1601/9999 with 7 missing, elap

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Imputing row 1/8198 with 12 missing, elapsed time: 25.287
Imputing row 101/8198 with 4 missing, elapsed time: 25.413
Imputing row 201/8198 with 5 missing, elapsed time: 25.534
Imputing row 301/8198 with 6 missing, elapsed time: 25.663
Imputing row 401/8198 with 13 missing, elapsed time: 25.778
Imputing row 501/8198 with 3 missing, elapsed time: 25.894
Imputing row 601/8198 with 24 missing, elapsed time: 26.037
Imputing row 701/8198 with 10 missing, elapsed time: 26.147
Imputing row 801/8198 with 15 missing, elapsed time: 26.251
Imputing row 901/8198 with 5 missing, elapsed time: 26.391
Imputing row 1001/8198 with 26 missing, elapsed time: 26.510
Imputing row 1101/8198 with 14 missing, elapsed time: 26.664
Imputing row 1201/8198 with 4 missing, elapsed time: 26.774
Imputing row 1301/8198 with 4 missing, elapsed time: 26.899
Imputing row 1401/8198 with 1 missing, elapsed time: 27.025
Imputing row 1501/8198 with 4 missing, elapsed time: 27.132
Imputing row 1601/8198 with 17 missing, elaps

In [41]:
# Computing derived variables

# Recompute P/F with no missing values...
reformat4t['PaO2_FiO2'] = reformat4t['paO2']/reformat4t['FiO2_1']

# Recompute SHOCK INDEX without NAN and INF
reformat4t['Shock_Index'] = reformat4t['HR']/reformat4t['SysBP']

reformat4t.loc[np.isinf(reformat4t['Shock_Index']), 'Shock_Index'] = np.nan
d = np.nanmean(reformat4t['Shock_Index'])
reformat4t['Shock_Index'].fillna(d, inplace=True)

shock_idx = reformat4t['Shock_Index'] >= np.quantile(reformat4t['Shock_Index'], 0.999)
reformat4t.loc[shock_idx, 'Shock_Index'] = np.quantile(reformat4t['Shock_Index'], 0.999)

# SOFA - at each timepoint we need (in this order):
# P/F,  MV,  PLT,  TOT_BILI,  MAP,  NORAD(max),  GCS, CR,  UO
s = reformat4t[['PaO2_FiO2', 'Platelets_count', 'Total_bili', 'MeanBP', 'max_dose_vaso', 'GCS', 'Creatinine', 'output_4hourly']].values
p = np.arange(5)
s1=np.array([s[:, 0]>400, (s[:, 0]>=300) & (s[:, 0]<400), (s[:, 0]>=200) & (s[:, 0]<300), (s[:, 0]>=100) & (s[:, 0]<200), s[:, 0]<100 ])   #count of points for all 6 criteria of sofa
s2=np.array([s[:, 1]>150, (s[:, 1]>=100) & (s[:, 1]<150), (s[:, 1]>=50) & (s[:, 1]<100), (s[:, 1]>=20) & (s[:, 1]<50), s[:, 1]<20 ])
s3=np.array([s[:, 2]<1.2, (s[:, 2]>=1.2) & (s[:, 2]<2), (s[:, 2]>=2) & (s[:, 2]<6), (s[:, 2]>=6) & (s[:, 2]<12), s[:, 2]>12 ])
s4=np.array([s[:, 3]>=70, (s[:, 3]<70) & (s[:, 3]>=65), (s[:, 3]<65), (s[:, 4]>0) & (s[:, 4]<=0.1), s[:, 4]>0.1 ])
s5=np.array([s[:,5]>14, (s[:, 5]>12) & (s[:, 5]<=14), (s[:, 5]>9) & (s[:, 5]<=12), (s[:, 5]>5) & (s[:, 5]<=9), s[:, 5]<=5])
s6=np.array([s[:,6]<1.2, (s[:,6 ]>=1.2) & (s[:, 6]<2), (s[:, 6]>=2) & (s[:, 6]<3.5), ((s[:, 6]>=3.5) & (s[:, 6]<5))|(s[:, 7]<84), (s[:, 6]>5)|(s[:, 7]<34)])

num_columns = reformat4t.shape[1]   # Number of variables in data
newcols_reformat4 = np.zeros((reformat4t.shape[0],7))
for i in range(reformat4t.shape[0]):
    t = max(p[s1[:,i]], default=0) + max(p[s2[:,i]], default=0) + max(p[s3[:,i]], default=0) + max(p[s4[:,i]], default=0) + max(p[s5[:,i]], default=0) + max(p[s6[:,i]], default=0)  #SUM OF ALL 6 CRITERIA
    if t > 0:
        newcols_reformat4[i,:] = [max(p[s1[:,i]], default=0), max(p[s2[:,i]], default=0), max(p[s3[:,i]], default=0),
            max(p[s4[:,i]], default=0), max(p[s5[:,i]], default=0), max(p[s6[:,i]], default=0), t]

# SIRS - at each timepoint |  need: temp HR RR PaCO2 WBC
s = reformat4t[['Temp_C', 'HR', 'RR', 'paCO2', 'WBC_count']].values

s1=np.array([(s[:,0]>=38) | (s[:,0]<=36)])   #count of points for all criteria of SIRS
s2=np.array([s[:,1]>90])
s3=np.array([(s[:,2]>=20) | (s[:,3]<=32)])
s4=np.array([(s[:,4]>=12) | (s[:,4]<4)])
newcols_sirs = (1*s1) + (1*s2) + (1*s3) + (1*s4)

# more IO corrections
reformat4t.loc[reformat4t['input_total'] < 0, 'input_total'] = 0
reformat4t.loc[reformat4t['input_4hourly'] < 0, 'input_4hourly'] = 0

# records values
reformat4t['SOFA'] = newcols_reformat4[:,-1]
reformat4t['SIRS'] = newcols_sirs[0]


<ipython-input-41-6499417073d0>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reformat4t['Shock_Index'].fillna(d, inplace=True)


In [43]:
print("save_intermediate =", pargs.save_intermediate)

save_intermediate = False


In [44]:
# Final MIMIC Table
out_file = project_path / "MIMICtable.csv"
MIMICtable = reformat4t.copy()

MIMICtable.to_csv(out_file, index=False)



In [61]:
out_file2 = project_path / 'sepsis_cohort.csv'
sepsis.to_csv(out_file2, index = False)

In [58]:
#len(reformat4t.columns)
len(MIMICtable['icustayid'].nunique())

TypeError: object of type 'int' has no len()

In [57]:
print(sepsis.shape[0])

17635
